## Fixing the issue with losses not matching in 2 codes


In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

import argparse
import os
import random


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml

from scipy import fftpack


In [2]:
%matplotlib widget

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
# import torchvision.utils as vutils
import numpy as np

import torch.fft

In [4]:
# sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
# import spec_loss as spc
# import post_analysis_pandas as post

In [5]:
############
### Numpy functions ### Not used in the code. Just to test the pytorch functions
############
def f_radial_profile(data, center=(None,None)):
    ''' Module to compute radial profile of a 2D image '''
    y, x = np.indices((data.shape)) # Get a grid of x and y values
    
    if center[0]==None and center[1]==None:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers
        
    # get radial values of every pair of points
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r = r.astype(np.int)
    
    # Compute histogram of r values
    tbin = np.bincount(r.ravel(), data.ravel())
    nr = np.bincount(r.ravel()) 
    radialprofile = tbin / nr
    
    return radialprofile[1:-1]

def f_compute_spectrum(arr,GLOBAL_MEAN=1.0):
    
    arr=((arr - GLOBAL_MEAN)/GLOBAL_MEAN)
    y1=np.fft.fft2(arr)
    y1=fftpack.fftshift(y1)

    y2=abs(y1)**2
    z1=f_radial_profile(y2)
    return(z1)

    
def f_compute_batch_spectrum(arr):
    batch_pk=np.array([f_compute_spectrum(i) for i in arr])
    return batch_pk


def f_image_spectrum(x,num_channels):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    print(x.shape)
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
#         print(i,arr.shape)
        batch_pk=f_compute_batch_spectrum(arr)
#         print(batch_pk)
        mean[i]=np.mean(batch_pk,axis=0)
        var[i]=np.var(batch_pk,axis=0)
    mean=np.array(mean)
    var=np.array(var)
    return mean,var

In [6]:

def f_torch_radial_profile(img, center=(None,None)):
    ''' Module to compute radial profile of a 2D image 
    Bincount causes issues with backprop, so not using this code
    '''
    
    y,x=torch.meshgrid(torch.arange(0,img.shape[0]),torch.arange(0,img.shape[1])) # Get a grid of x and y values
    if center[0]==None and center[1]==None:
        center = torch.Tensor([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0]) # compute centers

    # get radial values of every pair of points
    r = torch.sqrt((x - center[0])**2 + (y - center[1])**2)
    r= r.int()
    
#     print(r.shape,img.shape)
    # Compute histogram of r values
    tbin=torch.bincount(torch.reshape(r,(-1,)),weights=torch.reshape(img,(-1,)).type(torch.DoubleTensor))
    nr = torch.bincount(torch.reshape(r,(-1,)))
    radialprofile = tbin / nr
    
    return radialprofile[1:-1]


def f_torch_get_azimuthalAverage_with_batch(image, center=None): ### Not used in this code.
    """
    Calculate the azimuthally averaged radial profile. Only use if you need to combine batches

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
    batch, channel, height, width = image.shape
    # Create a grid of points with x and y coordinates
    y, x = np.indices([height,width])

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r = torch.tensor(np.hypot(x - center[0], y - center[1]))

    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (batch, channel,-1)))
    r_sorted = torch.gather(torch.reshape(r, (batch, channel, -1,)),2, ind)
    i_sorted = torch.gather(torch.reshape(image, (batch, channel, -1,)),2, ind)

    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[:,:,1:] - r_int[:,:,:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[2], (batch, -1))    # location of changes in radius
    rind=torch.unsqueeze(rind,1)
    nr = (rind[:,:,1:] - rind[:,:,:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin

    csum = torch.cumsum(i_sorted, axis=-1)
#     print(csum.shape,rind.shape,nr.shape)

    tbin = torch.gather(csum, 2, rind[:,:,1:]) - torch.gather(csum, 2, rind[:,:,:-1])
    radial_prof = tbin / nr

    return radial_prof


def f_get_rad(img):
    ''' Get the radial tensor for use in f_torch_get_azimuthalAverage '''
    
    height,width=img.shape[-2:]
    # Create a grid of points with x and y coordinates
    y, x = np.indices([height,width])
    
    center=[]
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r = torch.tensor(np.hypot(x - center[0], y - center[1]))
    
    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (-1,)))
    
    return r.detach(),ind.detach()


def f_torch_get_azimuthalAverage(image,r,ind):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
#     height, width = image.shape
#     # Create a grid of points with x and y coordinates
#     y, x = np.indices([height,width])

#     if not center:
#         center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

#     # Get the radial coordinate for every grid point. Array has the shape of image
#     r = torch.tensor(np.hypot(x - center[0], y - center[1]))

#     # Get sorted radii
#     ind = torch.argsort(torch.reshape(r, (-1,)))

    r_sorted = torch.gather(torch.reshape(r, ( -1,)),0, ind)
    i_sorted = torch.gather(torch.reshape(image, ( -1,)),0, ind)
    
    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[0], (-1,))    # location of changes in radius
    nr = (rind[1:] - rind[:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin
    
    csum = torch.cumsum(i_sorted, axis=-1)
    tbin = torch.gather(csum, 0, rind[1:]) - torch.gather(csum, 0, rind[:-1])
    radial_prof = tbin / nr

    return radial_prof

def f_torch_fftshift(real, imag):
    for dim in range(0, len(real.size())):
        real = torch.roll(real, dims=dim, shifts=real.size(dim)//2)
        imag = torch.roll(imag, dims=dim, shifts=imag.size(dim)//2)
    return real, imag

# def f_torch_compute_spectrum(arr,r,ind):
    
#     GLOBAL_MEAN=1.0
#     arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
#     y1=torch.rfft(arr,signal_ndim=2,onesided=False)
#     real,imag=f_torch_fftshift(y1[:,:,0],y1[:,:,1])    ## last index is real/imag part
#     y2=real**2+imag**2     ## Absolute value of each complex number
    
# #     print(y2.shape)
#     z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    
#     return z1

def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    
    y1=torch.fft.fftn(arr,dim=(-2,-1))
    real,imag=f_torch_fftshift(y1.real,y1.imag)    ## last index is real/imag part
    y2=real**2+imag**2     ## Absolute value of each complex number
    
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    
    return z1


def f_torch_compute_batch_spectrum(arr,r,ind):
    
    batch_pk=torch.stack([f_torch_compute_spectrum(i,r,ind) for i in arr])
    
    return batch_pk

def f_torch_image_spectrum(x,num_channels,r,ind):
    '''
    Data has to be in the form (batch,channel,x,y)
    '''
    
    mean=[[] for i in range(num_channels)]    
    var=[[] for i in range(num_channels)]    

    for i in range(num_channels):
        arr=x[:,i,:,:]
        batch_pk=f_torch_compute_batch_spectrum(arr,r,ind)
        mean[i]=torch.mean(batch_pk,axis=0)
#         sdev[i]=torch.std(batch_pk,axis=0)/np.sqrt(batch_pk.shape[0])
#         sdev[i]=torch.std(batch_pk,axis=0)
        var[i]=torch.var(batch_pk,axis=0)
    
    mean=torch.stack(mean)
    var=torch.stack(var)
        
    return mean,var



### Get images


In [7]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname)[:1000].transpose(0,1,2,3)
t_img=torch.from_numpy(img)
img.shape,t_img.shape

height,width=img.shape[-2:]
height,width

(128, 128)

In [8]:
mean_np,var_np=f_image_spectrum(img,1)

(1000, 1, 128, 128)


In [9]:
mean_np

array([[8.86095894e+04, 6.17923061e+04, 4.47503460e+04, 3.51597085e+04,
        2.81154319e+04, 2.29919677e+04, 1.86727547e+04, 1.56106763e+04,
        1.34797658e+04, 1.15257976e+04, 9.92852995e+03, 8.60875845e+03,
        7.49414036e+03, 6.56703667e+03, 5.76027553e+03, 5.11176780e+03,
        4.53719666e+03, 4.03557669e+03, 3.56441319e+03, 3.15770070e+03,
        2.80859481e+03, 2.51905774e+03, 2.26498943e+03, 2.00323965e+03,
        1.78629070e+03, 1.60299791e+03, 1.43996686e+03, 1.29375291e+03,
        1.16713012e+03, 1.05542881e+03, 9.47136378e+02, 8.57164956e+02,
        7.74504956e+02, 7.06768427e+02, 6.49635341e+02, 5.82564523e+02,
        5.33945159e+02, 4.86425736e+02, 4.49577822e+02, 4.14241110e+02,
        3.79101369e+02, 3.55290538e+02, 3.25002859e+02, 3.03822516e+02,
        2.80719548e+02, 2.62327633e+02, 2.47110733e+02, 2.28833887e+02,
        2.19269204e+02, 2.04426367e+02, 1.95022151e+02, 1.85564959e+02,
        1.73076708e+02, 1.68424720e+02, 1.58666063e+02, 1.535044

In [10]:
r,ind=f_get_rad(t_img)
mean_torch,var_torch=f_torch_image_spectrum(t_img,1,r,ind)

In [11]:
np.allclose(mean_np,mean_torch,rtol=1e-2),np.allclose(var_np,var_torch,rtol=1e-1)

(True, True)

##### Numpy and pytorch match!
Feb 4, 2021

In [15]:
def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    y1=torch.rfft(arr,signal_ndim=2,onesided=False)
    real,imag=f_torch_fftshift(y1[:,:,0],y1[:,:,1])    ## last index is real/imag part
    y2=real**2+imag**2     ## Absolute value of each complex number
    
#     print(y2.shape)
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    
    return z1
f_torch_compute_spectrum(t_img[0,0,:,:],r,ind)

<ipython-input-15-f87a720c5d1e>:5: UserWarning: The function torch.rfft is deprecated and will be removed in a future PyTorch release. Use the new torch.fft module functions, instead, by importing torch.fft and calling torch.fft.fft or torch.fft.rfft. (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370117127/work/aten/src/ATen/native/SpectralOps.cpp:590.)
  y1=torch.rfft(arr,signal_ndim=2,onesided=False)


tensor([7.9304e+04, 5.9219e+04, 2.9264e+04, 3.0270e+04, 2.3178e+04, 1.9839e+04,
        1.8123e+04, 1.7181e+04, 1.2256e+04, 9.6649e+03, 1.0016e+04, 8.0696e+03,
        7.3229e+03, 6.6355e+03, 5.8413e+03, 5.4688e+03, 4.1481e+03, 3.7668e+03,
        4.0713e+03, 2.7787e+03, 2.0897e+03, 2.2245e+03, 2.3392e+03, 1.9583e+03,
        1.6552e+03, 1.5430e+03, 1.2836e+03, 1.1684e+03, 1.2090e+03, 1.1141e+03,
        1.0758e+03, 8.5231e+02, 8.9757e+02, 7.8943e+02, 7.4062e+02, 5.4846e+02,
        4.7344e+02, 5.0072e+02, 4.4248e+02, 3.7514e+02, 3.4142e+02, 3.1127e+02,
        3.0085e+02, 3.1910e+02, 3.3995e+02, 2.8937e+02, 2.7358e+02, 2.4691e+02,
        2.5179e+02, 2.2854e+02, 2.0675e+02, 1.9259e+02, 2.0243e+02, 1.8627e+02,
        1.6845e+02, 1.6462e+02, 1.5684e+02, 1.4855e+02, 1.4567e+02, 1.3912e+02,
        1.3250e+02, 1.3640e+02, 1.2816e+02, 1.1948e+02, 1.0572e+02, 9.6471e+01,
        8.2032e+01, 6.9333e+01, 6.7000e+01, 5.4456e+01, 5.8791e+01, 5.1200e+01,
        4.6400e+01, 5.8057e+01, 4.9297e+

In [16]:
f_compute_spectrum(img[0,0,:,:],GLOBAL_MEAN=1.0)

array([7.93044775e+04, 5.92187458e+04, 2.92632039e+04, 3.02707217e+04,
       2.31769514e+04, 1.98385587e+04, 1.81240268e+04, 1.71809762e+04,
       1.22564506e+04, 9.66494253e+03, 1.00152396e+04, 8.06988950e+03,
       7.32282024e+03, 6.63569950e+03, 5.84087104e+03, 5.46882685e+03,
       4.14828832e+03, 3.76689817e+03, 4.07139020e+03, 2.77836809e+03,
       2.08977916e+03, 2.22444531e+03, 2.33917623e+03, 1.95845573e+03,
       1.65534980e+03, 1.54273280e+03, 1.28370677e+03, 1.16819342e+03,
       1.20923744e+03, 1.11411874e+03, 1.07582188e+03, 8.52234765e+02,
       8.97627462e+02, 7.89272089e+02, 7.40726941e+02, 5.48330201e+02,
       4.73606804e+02, 5.00708671e+02, 4.42393397e+02, 3.75179676e+02,
       3.41367374e+02, 3.11244553e+02, 3.00790316e+02, 3.19275384e+02,
       3.39897092e+02, 2.89407527e+02, 2.73605269e+02, 2.46800690e+02,
       2.51869766e+02, 2.28426231e+02, 2.06870006e+02, 1.92603530e+02,
       2.02389322e+02, 1.86229348e+02, 1.68418124e+02, 1.64680112e+02,
      

In [ ]:
def f_torch_compute_spectrum(arr,r,ind):
    
    GLOBAL_MEAN=1.0
    arr=(arr-GLOBAL_MEAN)/(GLOBAL_MEAN)
    
    y1=torch.fft.fftn(arr,dim=(-2,-1))
#     print(y1.shape)
    real,imag=f_torch_fftshift(y1.real,y1.imag)    ## last index is real/imag part
    
    y2=real**2+imag**2     ## Absolute value of each complex number
    
    z1=f_torch_get_azimuthalAverage(y2,r,ind)     ## Compute radial profile
    
    return z1




In [ ]:
r,ind=f_get_rad(t_img)
mean2_torch,var2_torch=f_torch_image_spectrum(t_img,1,r,ind)

In [ ]:
var2_torch

In [ ]:
var_torch